In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import IPython
import ipyvolume as ipv
import ipywidgets as ipw
import ipyparallel as ipp
import IPython.core.display as disp
from concurrent.futures import ThreadPoolExecutor

In [ ]:
import kale.aux_widgets

# NEWT Queue Widget 
- actually useful

In [ ]:
a = kale.aux_widgets.NEWTAuthWidget()
a

In [ ]:
q = kale.aux_widgets.QueueWidget(a)
q

# File Browser Widget
- a little bit useful

In [ ]:
kale.aux_widgets.FileBrowserWidget()

# SSH Widget
- Not really useful. Pretty lame, in fact.

In [ ]:
ssh_auth = kale.aux_widgets.SSHAuthWidget()
ssh_auth

In [ ]:
kale.aux_widgets.SSHTerminal(ssh_auth)

# Updating plot

Takes two arguments:
- `ar`: IPyParallel `AsyncResult`
- `y`: list of strings

The plot will poll `ar.data`, which is assumed to be a dict. The strings in `y` are assumed to be keys in `ar.data` whose corresponding values are 1D numpy arrays (or lists are probably okay). Those values will be plotted and labelled appropriately in the legend.

_More info_: When an IPyParallel task is launched, it immediately returns an `AsyncResult` with a `data` member dict. Calling `ipyparallel.datapub.publish_data` from the engine sends data back to the client (notebook) to populate this dict.

In [ ]:
c = ipp.Client()
lv = c.load_balanced_view()

def datapub_ex(N):
    import time
    import numpy as np
    from ipyparallel.datapub import publish_data
    
    y1 = []
    y2 = []
    
    for i in range(N):
        y1.append(np.random.rand())
        y2.append(np.random.rand())
        publish_data({
            'loss': y1,
            'acc': y2
        })
        time.sleep(1)

N = 10
ar = lv.apply(datapub_ex, N)
y = ['loss', 'acc']
up = kale.aux_widgets.UpdatingPlot(ar, y, xlim=[0, N-1], ylim=[0,1])
up

# Parameter Spans

Takes two required arguments:
- `compute_func`
- `vis_func`

and two important optional parameters:
- `product`
- `live`

The widget object, consisting of an empty `qgrid.QGridWidget` (widget wrapper for `pandas.DataFrame`) which will list parameter values and a blank `ipywidgets.Output` for visualization of results, is returned immediately by its constructor.

Once the widget is instantiated, `ParamSpanWidget.set_params(**params)` selects the parameter sets which will be run. Here, `params` is a dict of 1D lists whose keys are the names of the parameters required by `compute_func` and whose values specify the parameter values in one of are two modes:

- If `product=False`, then the lists must be all the same size, and no Cartesian product is taken. One parameter set will be run for each list index. This is like a random search.
- If `product=True`, then the lists can be of any length, and the Cartesian product will be taken, meaning that a grid search will be performed over all possible combinations.

Then, `ParamSpanWidget.submit_computations()` is called to actually submit the above-defined function calls via IPyParallel.

Then, for each parameter sets, the results of `compute_func` are passed to `vis_func` in one of two ways:

- If `live=False`, then the return value of `compute_func` is expected to be a dict whose keys are the same as the kwargs of `vis_func`. In this case, `vis_func` will only be called after task completion, and will be given the expanded dict returned by `compute_func` (i.e. `vis_func(**compute_func(**param_set))`).
- If `live=True`, then the only argument passed to `vis_func` will be the `ipyparallel.AsyncResult` returned by the `ipyparallel.Client.apply` call. (i.e. `vis_func(ipyparallel.Client().apply(compute_func, **param_set))`). In this case, `vis_func` will be called immediately upon task execution. This is suitable for use with `UpdatingPlot`. In this case, any return values from `compute_func` will be ignored.

*Note:* Presently, no data is implicitly saved to disk or DB on my end. I think IPyParallel captures everything by default (assuming you haven't started the controller with `--nodb`).
Of course, you can do so explicitly in `compute_func` or `vis_func` if you'd like.

## ParameterSpanWidget Example 
- `product=True, live=False`

In [ ]:
c = ipp.Client()

def exp_compute(N, mean, std, color):
    import numpy as np
    from datetime import datetime
    x = np.random.normal(loc=mean, scale=std, size=N)
    realmean = np.mean(x)
    realstd = np.std(x)
    return {
        'date': datetime.now().ctime(),
        'x': x,
        'N': N,
        'realmean': realmean,
        'realstd': realstd,
        'color': color
    }
    
def exp_viz(date, x, N, realmean, realstd, color):
    import matplotlib.pyplot as plt
    import seaborn as sns
    import warnings
    
    print("Computed at {}".format(date))
    plt.figure(figsize=[8,5])
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sns.distplot(x, color=color)
    plt.title(r'$N={}$, $\mu={:.2f}$, $\sigma={:.2f}$'.format(N, realmean, realstd))
    plt.show()
    print("Data: {}".format(x))

psw = kale.aux_widgets.ParamSpanWidget(
    compute_func=exp_compute,
    vis_func=exp_viz,
    product=True,
    live=False
)

psw.set_params(
    N = [100, 1000],
    mean = [5, 10],
    std = [1, 2],
    color = ['red', 'blue']
)

psw.submit_computations()
psw

For a useful example of these last two, see [cori-intml-examples/MNIST_Widgets.ipynb](https://github.com/OliverEvans96/cori-intml-examples/blob/master/MNIST_Widgets.ipynb)